# Prédiction de l'impact de sanctions économiques

## Notebook 1 - Modèle "naïf" (pas de feature engineering)

In [1]:
# import required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 80)
sns.set_style("whitegrid")
%matplotlib inline

In [2]:
# import the data

base_df = pd.read_csv("input/sanctions.csv")
print(base_df.shape)
# for col, typ in zip(base_df.columns, base_df.dtypes):
#     print(col, "=>", typ)
base_df.head(3)

(204, 39)


,Case no.b,Sender c,Sender 2c,Sender 3c,Targetd,Goal,US casee,US unilateral casef,Foreign policy goal categoryg,First yearh,Last yeari,Policy resultj,Sanctions contributionk,Success \nscorel,Companion policiesm,International cooperationn,International assistanceo,Cooperating international organizationp,International organization senderq,International organization sender & target membersr,Length (years)s,Prior relationst,Regime Type \n(DEMOC 1st year)u,Regime Type2 (Polity2 1st year)v,Regime Type3 \n(Scale 1st year)w,Political stability priorx,Political stability duringy,Cost to targetz,Cost to target (percent of GNP)aa,Cost to target per capitabb,Trade linkagecc,GNP \nratiodd,Health and stabilityee,Sanction typeff,Cost to sendergg,"GDP growth (percent, 5-year average)hh","Inflation (percent,\n 3-year average)ii",Target International Monetary Fund codejj,Country groupkk
0,14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,1918,4,3,12,R,4,A,--,0,0,4,1,5,2,2,"0,0","0,0",843,"7,1","12,58",9,1,3,"F,X,M",4,"3,6","2,9",134,1
1,17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,1918,2,2,4,--,1,--,--,0,0,1,2,5,1,2,"0,0","0,0",23,"0,8","0,44","20,5",13,3,X,2,"5,4","25,6",158,1
2,18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,1,2,2,"R,Q",4,--,--,0,0,2,1,1,-1,2,"0,1","0,0",446,"4,1","2,49","18,5",1,1,"F,X,M",3,n.a.,n.a.,922,2


In [3]:
# Drop information that is unknown at decision time

unknowns = [
    "Last yeari",
    "Length (years)s",
    "Political stability duringy",
    "Cost to targetz",
    "Cost to target (percent of GNP)aa",
    "Cost to target per capitabb",
    "Cost to sendergg",
    "Target International Monetary Fund codejj"
]

df = base_df.drop(unknowns, axis=1)

print(df.shape)
df.head(3)

(204, 31)


,Case no.b,Sender c,Sender 2c,Sender 3c,Targetd,Goal,US casee,US unilateral casef,Foreign policy goal categoryg,First yearh,Policy resultj,Sanctions contributionk,Success \nscorel,Companion policiesm,International cooperationn,International assistanceo,Cooperating international organizationp,International organization senderq,International organization sender & target membersr,Prior relationst,Regime Type \n(DEMOC 1st year)u,Regime Type2 (Polity2 1st year)v,Regime Type3 \n(Scale 1st year)w,Political stability priorx,Trade linkagecc,GNP \nratiodd,Health and stabilityee,Sanction typeff,"GDP growth (percent, 5-year average)hh","Inflation (percent,\n 3-year average)ii",Country groupkk
0,14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,4,3,12,R,4,A,--,0,0,1,5,2,2,"0,0",9,1,3,"F,X,M","3,6","2,9",1
1,17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,2,2,4,--,1,--,--,0,0,2,5,1,2,"0,0","20,5",13,3,X,"5,4","25,6",1
2,18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1,2,2,"R,Q",4,--,--,0,0,1,1,-1,2,"0,1","18,5",1,1,"F,X,M",n.a.,n.a.,2
